In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [5]:
data=pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Measles/nigeria.csv",index_col='Year-Month', parse_dates=['Year-Month'], dayfirst=False)


In [6]:
data

,Cases
Year-Month,
2011-01-01,4539
2011-02-01,4025
2011-03-01,4533
2011-04-01,2520
2011-05-01,1057
...,...
2023-11-01,200
2023-12-01,134
2024-01-01,2132


In [7]:
dt=data.rename({'Cases' : 'cases'}, axis=1)
dt

,cases
Year-Month,
2011-01-01,4539
2011-02-01,4025
2011-03-01,4533
2011-04-01,2520
2011-05-01,1057
...,...
2023-11-01,200
2023-12-01,134
2024-01-01,2132


In [8]:
df= dt.groupby("Year-Month").sum()

In [9]:
train = dt.iloc[:147]
test = dt.iloc[147:]

In [10]:
len(test)

12

In [11]:
train

,cases
Year-Month,
2011-01-01,4539
2011-02-01,4025
2011-03-01,4533
2011-04-01,2520
2011-05-01,1057
...,...
2022-11-01,238
2022-12-01,284
2023-01-01,2013


In [12]:
train= train.groupby("Year-Month").sum()
test= test.groupby("Year-Month").sum()

In [13]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [15]:
hwmodel= ExponentialSmoothing(train["cases"], trend= "add", seasonal= "mul", seasonal_periods= 12,initialization_method="heuristic",use_boxcox=False,damped=False).fit(
    smoothing_level=1,
    smoothing_trend=0.1,
    smoothing_seasonal=0.1)
hwmodel.summary()

<ipython-input-15-ab0bcec4722d>:1: FutureWarning: the 'damped' keyword is deprecated, use 'damped_trend' instead.
  hwmodel= ExponentialSmoothing(train["cases"], trend= "add", seasonal= "mul", seasonal_periods= 12,initialization_method="heuristic",use_boxcox=False,damped=False).fit(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:210: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)


Dep. Variable:,cases,No. Observations:,147
Model:,ExponentialSmoothing,SSE,145100064.277
Optimized:,False,AIC,2060.968
Trend:,Additive,BIC,2108.815
Seasonal:,Multiplicative,AICC,2066.311
Seasonal Periods:,12,Date:,"Fri, 19 Jul 2024"
Box-Cox:,False,Time:,22:25:09
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,1.0000000,alpha,False
smoothing_trend,0.1000000,beta,False


In [16]:
start = len(train)
end = len(train)+len(test)-1

prd1 = hwmodel.predict(start= start, end= end)

prd1.index = df.index[start: end+1]

print(prd1)

Year-Month
2023-04-01    1301.423709
2023-05-01     580.482882
2023-06-01     328.060290
2023-07-01     184.290279
2023-08-01      96.620260
2023-09-01      75.793589
2023-10-01      61.944436
2023-11-01      61.651374
2023-12-01      44.874455
2024-01-01     248.056760
2024-02-01     194.258641
2024-03-01      58.096384
dtype: float64


In [19]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(test["cases"],prd1))
#print('Test RMSE: %.4f' % error)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test["cases"],prd1)
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(test["cases"],prd1)
#print("Mean Squared Logarithmic Error (MSLE):", msle)

from sklearn.metrics import mean_absolute_percentage_error
#print('Mean Absolute Percentage Error (MAPE):',mean_absolute_percentage_error(valid["Cases"],predictions))

print('RMSE: %.2f' % error)
def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape = mean_absolute_percentage_error(test["cases"],prd1)
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)

RMSE: 704.88
MAPE: 92.99%
MAE:  488.64
MSLE: 1.60
